In [ ]:
#ler planilha excel,pegar nome, termos banidos, preço minimo , preço maximo 
#abrir navegador, google shopping, pegar todos os resultados que tenham preço dentro da faixa e sejam o os produtos corretos
#o mesmo para o buscape
#enviar um email com uma tabela com os itens e preços encontrados junto com o link de compra

In [170]:
import serpapi
import pandas as pd
import os 
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


caminho = os.getcwd()
caminho_arquivo = caminho+ r"\buscas.xlsx"
tabela = pd.read_excel(caminho_arquivo) # base de dados
display(tabela)

servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--disable-notifications')


nome = tabela.loc[0,'Nome']
termos_banidos = tabela.loc[0,'Termos banidos'].split(" ")
prec_min = tabela.loc[0,'Preço mínimo']
prec_max = tabela.loc[0,'Preço máximo']

teste = 'meia-noite'



,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 14 128gb,mini watch,3500,4000
1,rtx 3080 ti,zota galax,4000,4500


In [171]:
def pesquisaGoogle(nome,termos_banidos,prec_min,prec_max):
    api_key = 'chave da api' #chave do serpapi
    client = serpapi.Client(api_key=api_key)
    
    lista_ofertas = []
    resultados = client.search({
        'engine': 'google_shopping',
        'q':nome, # nome do produto
        'gl': 'br', # apenas buscas no Brasil
    })

    for produto in resultados['shopping_results']:
        preco = produto['extracted_price'] #preco do produto
        nome =  produto['title'].lower() # nome do produto minusculo
        link = produto['link'] # link do produto
        if prec_min<=preco<=prec_max:
            for termo in termos_banidos:
                if termo not in nome:
                    lista_ofertas.append((nome,preco,link))
    
    return lista_ofertas
                    


In [172]:
def pesquisaBuscape(nome,termos_banidos,prec_min,prec_max):
    navegador = webdriver.Chrome(options=options)
    navegador.get("https://www.buscape.com.br/")
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(nome, Keys.ENTER)

    lista_ofertas = []
    lista_prod = navegador.find_elements(By.CLASS_NAME,'ProductCard_ProductCard_Inner__gapsh')
    nome_prod = navegador.find_elements(By.CLASS_NAME,'ProductCard_ProductCard_Name__U_mUQ')
    prec_prod = navegador.find_elements(By.CLASS_NAME,'Text_MobileHeadingS__HEz7L')

    for i,nome in enumerate(nome_prod):
        preco = prec_prod[i].text
        preco = float(preco.replace("R$","").replace(" ","").replace(".","").replace(",","."))
        nome =  nome_prod[i].text
        nome = nome.lower()
        verificacao = True
        
        for termo in termos_banidos:
            if termo in nome:
                verificacao = False

        if prec_min<=preco<=prec_max and verificacao==True:
            link = lista_prod[i].get_attribute('href')
            lista_ofertas.append((nome,preco,link))

    return lista_ofertas

In [173]:
lista_google = pesquisaGoogle(nome,termos_banidos,prec_min,prec_max)
lista_buscape = pesquisaBuscape(nome,termos_banidos,prec_min,prec_max)
print(lista_google)
print(lista_buscape)

The chromedriver version (127.0.6533.99) detected in PATH at c:\Users\Guilherme\AppData\Local\Programs\Python\Python312\chromedriver.exe might not be compatible with the detected chrome version (128.0.6613.86); currently, chromedriver 128.0.6613.86 is recommended for chrome 128.*, so it is advised to delete the driver in PATH and retry


[('apple iphone 14 - 128gb - meia-noite.', 3878.99, 'https://www.amazon.com.br/Apple-iPhone-14-128-Cinza-Escuro/dp/B00CLOZBHK?source=ps-sl-shoppingads-lpcontext&ref_=fplfs&psc=1&smid=A1ZZFT5FULY4LN'), ('apple iphone 14 - 128gb - meia-noite.', 3878.99, 'https://www.amazon.com.br/Apple-iPhone-14-128-Cinza-Escuro/dp/B00CLOZBHK?source=ps-sl-shoppingads-lpcontext&ref_=fplfs&psc=1&smid=A1ZZFT5FULY4LN'), ('vitrine iphone 14 preto 128gb', 3779.99, 'https://www.carrefour.com.br/vitrine-----iphone---14---preto--128gb-mp936555183/p'), ('vitrine iphone 14 preto 128gb', 3779.99, 'https://www.carrefour.com.br/vitrine-----iphone---14---preto--128gb-mp936555183/p'), ('vitrine iphone 14 azul 128 gb', 3779.99, 'https://www.carrefour.com.br/vitrine-iphone--14--azul--128-gb-mp936556529/p'), ('vitrine iphone 14 azul 128 gb', 3779.99, 'https://www.carrefour.com.br/vitrine-iphone--14--azul--128-gb-mp936556529/p'), ('usado: iphone 14 128gb azul excelente - trocafone - apple', 3507.94, 'https://www.magazinelui